#1. Preparation

Runtime => GPU

In [ ]:
!wget https://sid.erda.dk/public/archives/ff17dc924eba88d5d01a807357d6614c/FullIJCNN2013.zip

In [ ]:
!unzip FullIJCNN2013.zip

In [ ]:
!ls -al FullIJCNN2013

In [ ]:
import pandas as pd
import os
import cv2

In [ ]:
test_percentage = 0.2
file_name = './FullIJCNN2013/gt.txt'

In [ ]:
annotation = pd.read_csv(file_name, names=['ImageId', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2', 'ClassId'], sep=';')

In [ ]:
annotation.head()

In [ ]:
Prohibitory = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 15, 16]
Danger = [11, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
Mandatory = [33, 34, 35, 36, 37, 38, 39, 40]
Other = [6, 12, 13, 14, 17, 32, 41, 42]

categories = ['Prohibitory', 'Danger', 'Mandatory', 'Other']

In [ ]:
annotation['CategoryId'] = 0

In [ ]:
annotation.loc[annotation['ClassId'].isin(Prohibitory), 'CategoryId'] = 0
annotation.loc[annotation['ClassId'].isin(Danger), 'CategoryId'] = 1
annotation.loc[annotation['ClassId'].isin(Mandatory), 'CategoryId'] = 2
annotation.loc[annotation['ClassId'].isin(Other), 'CategoryId'] = 3

In [ ]:
annotation['centerX'] = (annotation['Roi.X2']+annotation['Roi.X1']) / 2 
annotation['centerY'] = (annotation['Roi.Y2']+annotation['Roi.Y1']) / 2 
annotation['width'] = (annotation['Roi.X2'] - annotation['Roi.X1']) 
annotation['height'] = (annotation['Roi.Y2'] - annotation['Roi.Y1']) 

In [ ]:
annotation.head()

In [ ]:
image_path = './FullIJCNN2013'

In [ ]:
os.chdir(image_path)

In [ ]:
!ls -al

In [ ]:
paths = []
for current_dir, dirs, files in os.walk('.'):
    for file in files:
        if file.endswith('.ppm'):
            image_ppm = cv2.imread(file)
            h, w = image_ppm.shape[:2]
            image_name = file.split('.')[0]
            txt_file = image_name + '.txt'
            image_jpg = image_name + '.jpg'
            cv2.imwrite(image_jpg, image_ppm)
            txt = open(txt_file, 'w')
            objects = annotation.loc[annotation['ImageId'] == file].copy()
            for index, data in objects.iterrows():
                line = str(data['CategoryId']) + ' ' + str(data['centerX']/w) + ' ' + str(data['centerY']/h) + ' ' + str(data['width']/w) + ' ' + str(data['height']/h) + '\n'
                txt.write(line)
            image_path = image_jpg + '\n'
            paths.append(image_path)
            print(image_jpg, 'is created')

In [ ]:
num_test = int(len(paths) * test_percentage)
print(len(paths))
print(num_test)

In [ ]:
paths_test = paths[:num_test]
paths_train = paths[num_test:]

In [ ]:
with open('train.txt', 'w') as train_txt:
    for path in paths_train:
        train_txt.write(path)
    print('[train.txt] is created')

In [ ]:
with open('test.txt', 'w') as test_txt:
    for path in paths_test:
        test_txt.write(path)
    print('[test.txt] is created')

In [ ]:
with open('classes.names', 'w') as names:
    for category in categories:
        names.write(category)
    print('[classes.names] is created')

In [ ]:
with open('traffic.data', 'w') as data:
    data.write('classes = ' + str(len(categories)) + '\n')
    data.write('train = ' + 'train.txt' + '\n')
    data.write('valid = ' +  'test.txt' + '\n')
    data.write('names = ' +  'classes.names' + '\n')
    data.write('backup = backup')
    print('[custom_data.data] is created')

In [ ]:
!mkdir backup

In [ ]:
# traffic-sign-yolo.cfg
# darknet

from google.colab import files
files.upload()

In [ ]:
!chmod +x ./darknet

In [ ]:
# Download Pretrained Convolutional Weights
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
!ls -al

In [ ]:
# Train The Model
!./darknet detector train traffic.data traffic-sign-yolo.cfg darknet53.conv.74 -dont_show